In [ ]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import ptitprince as pt
from scipy.io import loadmat
import matplotlib.pyplot as plt
# pandas==1.5.3
# seaborn==0.11
# ptitprince==0.2.7
# matplotlib==3.8.4


def plot(overlaps: np.ndarray):
    data = {
        r'$|v_1\rangle$': overlaps[:, 0],
        r'$|v_2\rangle$': overlaps[:, 1],
        r'$|v_3\rangle$': overlaps[:, 2],
        r'$|v_4\rangle$': overlaps[:, 3],
    }
    colors = ['#6FAE45', '#589CD6', '#8F08A6', '#AB2A3C']
    df = pd.DataFrame(data)
    pt.half_violinplot(data=df, scale='count', width=0.6, \
        inner=None, linewidth=1.2, palette=colors)
    sns.stripplot(data=df, jitter=True, palette=colors, size=1)
    overlaps_max = np.ceil(overlaps.max() * 10) / 10
    plt.yticks(np.linspace(0, overlaps_max, 11))
    plt.ylabel('Overleaps')
    plt.show()


path = r'C:\GitHub\QuditVGON\mats'
# path = r'\\wsl.localhost\Fedora\home\ghost\QuditVGON\mats'
for name in sorted(os.listdir(path), reverse=True):
    match = re.search(r'(VGON_nqd\d+_\d{8}_\d{6}).mat', name)
    if match and '20241025' in name:
        load = loadmat(f'{path}/{match.group(1)}.mat')
        energy = load['energy'].item()
        n_qudits = int(re.search(r'nqd(\d+)', name).group(1))
        if 'cos_sim' in load:
            cos_sim = load['cos_sim'].item()
            if 'fidelity' in load:
                fidelity = load['fidelity'].item()
                info_str = f'Fidelity: {fidelity:.8f}, Cos_Sim: {cos_sim:.8f}'
            else:
                info_str = f'Cos_Sim: {cos_sim:.8f}'
        elif 'cos_sim_max' in load and 'cos_sim_mean' in load:
            cos_sim_max = load['cos_sim_max'].item()
            cos_sim_mean = load['cos_sim_mean'].item()
            info_str = f'Cos_Sim: {cos_sim_max:.8f}, {cos_sim_mean:.8f}'
        else:
            fidelity = load['fidelity'].item()
            info_str = f'Fidelity: {fidelity:.8f}'
        n_train = (load['n_train'] if 'n_train' in load else load['n_iter']).item()
        if 'overlaps' in load:
            print(f'{name}, Energy: {energy:.8f}, {info_str}, {n_train}')
            overlaps = load['overlaps']
            plot(overlaps)
        else:
            info_str += ', No overlaps'
            print(f'{name}, Energy: {energy:.8f}, {info_str}, {n_train}')

In [ ]:
import scipy.io as sio
import sys, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ptitprince as pt

# load data
name = 'MG_re_plot'
data = sio.loadmat(os.path.join(sys.path[0], 'Data', name + '.mat'))
data = data['m']

data = {
    r'$|v_1 \rangle$': data[:, 0],
    r'$|v_2 \rangle$': data[:, 1],
    r'$|v_3 \rangle$': data[:, 2],
    r'$|v_4 \rangle$': data[:, 3],
    r'$|v_5 \rangle$': data[:, 4],
}
# colors = ['#B5C6E7', '#BED8EF', '#C6E1B8', '#AB2A3C', '#660874']
colors = ['#6FAE45', '#589CD6', '#4474C4', '#660874', '#AB2A3C']
# Convert data to DataFrame
df = pd.DataFrame(data)

# Calculate statistics using DataFrame
stats = df.describe()
print(stats)

# Create raincloud plot
fig, ax = plt.subplots()
ax = pt.half_violinplot(data=df, inner=None, width=1, palette=colors, edgecolor='white')
ax = sns.stripplot(data=df, jitter=True, edgecolor='none', size=1.2, palette=colors)
# ax = sns.boxplot(data=df,width = 0.25, zorder = 10,  whiskerprops = {'linewidth':1.5, 'zorder':10}, boxprops = {'facecolor':'none', 'zorder':10} ,palette=colors)

plt.ylim(-0.1, 1)
# Add grid to the plot
plt.grid(True)
# Set plot labels and title
ax.set_ylabel('Overlap')

# plt.title('Raincloud Plot')

# Show the plot
# plt.show()

# Calculate statistics for each group

# savefig
path = os.path.join(sys.path[0], 'Image', '1MG_Box.pdf')
plt.savefig(path, format='pdf', bbox_inches='tight')
path = os.path.join(sys.path[0], 'Image', '1MG_Box.png')
plt.savefig(path, format='jpg', bbox_inches='tight')